In [1]:
import sys
sys.path.append('../../..')
from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.agents import Agent
from hydradx.model.processing import get_kraken_orderbook, get_omnipool_data, get_stableswap_data, get_orderbooks_from_file, get_binance_orderbook
from hydradx.model.processing import get_omnipool, get_centralized_market
from hydradx.model.amm.stableswap_amm import StableSwapPoolState 
from hydradx.model.amm.centralized_market import CentralizedMarket
import hydradx.model.amm.arbitrage_agent_general as arb

from pprint import pprint

import json

Set `get_live_markets = True` to get live markets from Kraken. Set to `False` to use order books from file.

In [2]:
get_live_markets = True
archive_data = False

In [3]:
# 2: Acala DAI
# 3: Acala WBTC
# 4: Acala WETH
# 5: DOT
# 9: ASTR
# 10: native USDT
# 11: iBTC
# 12: ZTG
# 13: CFG
# 14: BNC
# 15: vDOT
# 16: GLMR
# 17: INTR
# 18: Wormhole DAI
# 19: Wormhole WBTC
# 20: Wormhole WETH
# 100: 4-Pool

In [4]:
exchanges = {}
# load Omnipool (with stableswap subpools)
print('loading omnipool...')
RPC="wss://rpc.hydradx.cloud"
            
if get_live_markets:
    omnipool = get_omnipool()
else:
    omnipool = load_omnipool()

exchanges['omnipool'] = omnipool
exchanges['4-Pool'] = omnipool.sub_pools['4-Pool']
exchanges['2-Pool'] = omnipool.sub_pools['2-Pool']

loading omnipool...


In [5]:
# load centralized markets
print('loading centralized markets...')
# arb_file = "arbconfig.txt"
arb_files = ["arbconfig4.txt"]

cfg = []
for arb_file in arb_files:
    with open('config/' + arb_file, 'r') as json_file:
        cfg += json.load(json_file)

for d in cfg:
    for ex in d['exchanges']:
        d['exchanges'][ex] = tuple(d['exchanges'][ex])

# pprint(cfg)
    
for exchange, trade_fee in {'kraken': 0.0016, 'binance': 0.0010}.items():
    if get_live_markets:
        exchanges[exchange] = get_centralized_market(
            config=cfg, 
            exchange_name=exchange, 
            trade_fee=trade_fee,
            archive=archive_data
        )
    else:
        exchanges[exchange] = CentralizedMarket(
            order_book=get_orderbooks_from_file("archive/")[exchange],
            unique_id=exchange,
            trade_fee=trade_fee
        )

loading centralized markets...


In [6]:
print('calculating arbitrage...')
all_swaps = arb.get_arb_swaps(exchanges, cfg)

calculating arbitrage...


In [7]:
test_exchanges = {ex_name: ex.copy() for ex_name, ex in exchanges.items()}
init_agent = Agent(
    holdings={tkn: 100000000 for tkn in [tkn for ex in exchanges.values() for tkn in ex.asset_list]}
)
test_agent = init_agent.copy()
arb.execute_arb(exchanges, test_agent, all_swaps)

equivalency_map = {
    'WETH': 'ETH',
    'XETH': 'ETH',
    'XXBT': 'BTC',
    'WBTC': 'BTC',
    'ZUSD': 'USD',
    'USDT': 'USD',
    'USDT10': 'USD',
    'USDT23': 'USD',
    'USDC': 'USD',
    'DAI': 'USD',
    'iBTC': 'BTC',
    'XBT': 'BTC',
}

profit = arb.calculate_profit(init_agent, test_agent, equivalency_map)

print("per-token profit")
print(profit)
print("total profit at Binance spot")
print(exchanges['binance'].value_assets(profit, equivalency_map))

per-token profit
{'4-Pool': 0, 'HDX': 0.0, 'USD': 4.404955327510834, 'ETH': 0, '2-Pool': 0, 'GLMR': 0, 'BNC': 0.0, 'vDOT': 0, 'CFG': 0.0, 'DOT': 0, 'PHA': 0, 'ZTG': 0, 'INTR': 0, 'ASTR': 0, 'BTC': 0}
total profit at Binance spot
4.404955327510834
